# Data Science for Business - Multilayer Perceptron (MLP) on Ames Housing with Pytorch

## Initialize notebook
Load required packages. Set up workspace, e.g., set theme for plotting and initialize the random number generator.

In [1]:
import warnings
warnings.simplefilter('ignore')

import numpy as np
import pandas as pd

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import OneHotEncoder
from sklearn.metrics import r2_score, root_mean_squared_error

import torch
import torch.nn as nn
import torch.optim as optim
from torchsummary import summary


In [2]:
torch.manual_seed(42)

## Problem description

Ask a home buyer to describe their dream house, and they probably won't begin with the height of the basement ceiling or the proximity to an east-west railroad. But this dataset proves that much more influences price negotiations than the number of bedrooms or a white-picket fence. With 76 explanatory variables describing (almost) every aspect of residential homes in Ames, Iowa, this dataset challenges you to predict the final price of each home. More: <https://www.kaggle.com/c/house-prices-advanced-regression-techniques>


## Load data

Load training data from CSV file.

In [3]:
data = pd.read_csv('https://raw.githubusercontent.com/olivermueller/ds4b-2024/refs/heads/main/Session_08/ameshousing.csv')

In [4]:
data.head()

,MSSubClass,MSZoning,LotFrontage,LotArea,Street,Alley,LotShape,LandContour,Utilities,LotConfig,...,ThreeSsnPorch,ScreenPorch,PoolArea,Fence,MiscVal,MoSold,YrSold,SaleType,SaleCondition,SalePrice
0,20,RL,141,31770,Pave,none,IR1,Lvl,AllPub,Corner,...,0,0,0,none,0,5,2010,WD,Normal,215000
1,20,RH,80,11622,Pave,none,Reg,Lvl,AllPub,Inside,...,0,120,0,MnPrv,0,6,2010,WD,Normal,105000
2,20,RL,81,14267,Pave,none,IR1,Lvl,AllPub,Corner,...,0,0,0,none,12500,6,2010,WD,Normal,172000
3,20,RL,93,11160,Pave,none,Reg,Lvl,AllPub,Corner,...,0,0,0,none,0,4,2010,WD,Normal,244000
4,60,RL,74,13830,Pave,none,IR1,Lvl,AllPub,Inside,...,0,0,0,MnPrv,0,3,2010,WD,Normal,189900


## Prepare data

First, we will remove some columns that are not useful for our task.

In [5]:
data = data.drop(['YrSold', 'MoSold', 'SaleCondition', 'SaleType'], axis=1)

Next, we will split the data into features (*X*) and labels (*y*) and into training (*X_train, y_train*) and test (*X_test, y_test*) sets.

In [6]:
X = data.drop("SalePrice", axis=1)
y = data["SalePrice"]
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

Finally, we will do some feature engineering. It is important to use only information from the training set for feature engineering, and the mechanistically repeat these steps on the test set.

Typically, feature engineering depends strongly on the datatype of the variables. Hence, we will first determine which variables are categorical and which are numerical. Subsequentally, we will transform these variables seperately.

In [7]:
categorical_features = X_train.select_dtypes(include='object').columns
numerical_features = X_train.select_dtypes(exclude='object').columns

The categorical variables must be transformed into numerical representations, e.g., by one-hot encdoing them.

In [8]:
enc = OneHotEncoder(handle_unknown='ignore', sparse_output=False)
enc.fit(X_train[categorical_features])

X_train_cat = enc.transform(X_train[categorical_features])
X_test_cat = enc.transform(X_test[categorical_features])

X_train_cat = pd.DataFrame(X_train_cat, columns=enc.get_feature_names_out(categorical_features))
X_test_cat = pd.DataFrame(X_test_cat, columns=enc.get_feature_names_out(categorical_features))

In [9]:
X_train_cat.head()

,MSZoning_A(agr),MSZoning_C(all),MSZoning_FV,MSZoning_I(all),MSZoning_RH,MSZoning_RL,MSZoning_RM,Street_Grvl,Street_Pave,Alley_Grvl,...,GarageCond_TA,GarageCond_none,PavedDrive_N,PavedDrive_P,PavedDrive_Y,Fence_GdPrv,Fence_GdWo,Fence_MnPrv,Fence_MnWw,Fence_none
0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,...,1.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0
1,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,...,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0
2,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,...,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0
3,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,...,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0
4,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,...,0.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0


The numerical variables will be standardized, that is, we will subtract the mean and divide by the standard deviation. This is especially important for LASSO, as all coefficients need to be comparable in terms of units and magnitudes.

In [10]:
scaler = StandardScaler()
scaler.fit(X_train[numerical_features])

X_train_num = scaler.transform(X_train[numerical_features])
X_test_num = scaler.transform(X_test[numerical_features])

X_train_num = pd.DataFrame(X_train_num, columns=numerical_features)
X_test_num = pd.DataFrame(X_test_num, columns=numerical_features)

In [11]:
X_train_num.head()

,MSSubClass,LotFrontage,LotArea,OverallQual,OverallCond,YearBuilt,YearRemodAdd,MasVnrArea,BsmtFinSF1,BsmtFinSF2,...,Fireplaces,GarageCars,GarageArea,WoodDeckSF,OpenPorchSF,EnclosedPorch,ThreeSsnPorch,ScreenPorch,PoolArea,MiscVal
0,-0.871817,0.667826,0.033810,0.673941,-0.526415,0.181084,-0.381277,0.531409,-0.978369,-0.293998,...,0.614268,0.339813,0.047615,-0.753959,-0.695967,-0.369038,-0.098812,-0.286876,-0.067407,-0.09315
1,0.062906,-1.717704,2.307082,-0.766750,-0.526415,-0.115603,-0.814347,-0.569155,-0.427633,4.191480,...,-0.917482,0.339813,0.325180,3.139485,-0.695967,-0.369038,-0.098812,3.744734,-0.067407,-0.09315
2,0.763949,0.369635,-0.035514,-1.487095,-0.526415,-0.280430,-1.054941,-0.569155,-0.978369,-0.293998,...,-0.917482,0.339813,-0.032361,-0.753959,-0.695967,-0.369038,-0.098812,-0.286876,-0.067407,-0.09315
3,0.763949,0.071444,-0.363746,-1.487095,-0.526415,-0.708978,-1.632368,-0.569155,-0.978369,-0.293998,...,-0.917482,0.339813,-0.229950,-0.753959,-0.695967,-0.369038,-0.098812,-0.286876,-0.067407,-0.09315
4,3.100756,0.160901,-0.310697,-1.487095,0.378216,-1.664971,-1.632368,-0.569155,-0.978369,-0.293998,...,-0.917482,-2.337571,-2.205835,-0.753959,-0.695967,1.839369,-0.098812,-0.286876,-0.067407,-0.09315


Let's fuse the enginnered categorical and numerical variables again.

In [12]:
X_train = pd.concat([X_train_num, X_train_cat], axis=1)
X_test = pd.concat([X_test_num, X_test_cat], axis=1)

In [13]:
X_train

,MSSubClass,LotFrontage,LotArea,OverallQual,OverallCond,YearBuilt,YearRemodAdd,MasVnrArea,BsmtFinSF1,BsmtFinSF2,...,GarageCond_TA,GarageCond_none,PavedDrive_N,PavedDrive_P,PavedDrive_Y,Fence_GdPrv,Fence_GdWo,Fence_MnPrv,Fence_MnWw,Fence_none
0,-0.871817,0.667826,0.033810,0.673941,-0.526415,0.181084,-0.381277,0.531409,-0.978369,-0.293998,...,1.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0
1,0.062906,-1.717704,2.307082,-0.766750,-0.526415,-0.115603,-0.814347,-0.569155,-0.427633,4.191480,...,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0
2,0.763949,0.369635,-0.035514,-1.487095,-0.526415,-0.280430,-1.054941,-0.569155,-0.978369,-0.293998,...,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0
3,0.763949,0.071444,-0.363746,-1.487095,-0.526415,-0.708978,-1.632368,-0.569155,-0.978369,-0.293998,...,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0
4,3.100756,0.160901,-0.310697,-1.487095,0.378216,-1.664971,-1.632368,-0.569155,-0.978369,-0.293998,...,0.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2339,3.100756,4.097025,3.909976,-0.766750,-0.526415,-0.049673,0.292388,-0.569155,1.337375,-0.293998,...,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
2340,0.062906,-1.717704,-0.295416,0.673941,-0.526415,1.038181,0.869815,-0.569155,-0.978369,-0.293998,...,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0
2341,0.062906,0.190720,-0.166086,-0.046404,-0.526415,1.071146,0.917934,-0.569155,0.134161,-0.293998,...,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0
2342,-0.170774,0.697645,-0.350328,-0.766750,-0.526415,-1.664971,-1.632368,-0.569155,-0.182124,-0.293998,...,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0


## Neural Network

### Data loaders

The first thing we have to do is to transform the data into Pytorch tensors. This is done by the `torch.tensor` function.

In [14]:
# Convert to PyTorch tensors
X_train_tensor = torch.tensor(X_train.to_numpy(), dtype=torch.float32)
X_test_tensor = torch.tensor(X_test.to_numpy(), dtype=torch.float32)
y_train_tensor = torch.tensor(y_train.to_numpy(), dtype=torch.float32).unsqueeze(1)
y_test_tensor = torch.tensor(y_test.to_numpy(), dtype=torch.float32).unsqueeze(1)

Next, we create a data loader for the training data, using the `torch.utils.data.DataLoader` function. The data loader allows to read the data in a streaming fashion during training. This is actually not needed for our tiny dataset, but normally one uses PyTorch with much larger datasets that do not fit into main memory.

In [15]:
batch_size = 32

train_loader = torch.utils.data.DataLoader(
    dataset=torch.utils.data.TensorDataset(X_train_tensor, y_train_tensor),
    batch_size=batch_size,
    shuffle=True
)

### Network architecture

Now we specify the architecture of the neural network. We will use a simple feedforward neural network with one hidden layers. The input layer has the same number of neurons as we have features, the output layer has one neuron, as we want to do regression. The hidden layer has 256 neurons. We use the ReLU activation function for the hidden layer.

To specify the architecture, we create a class that inherits from `torch.nn.Module` and has two standard methods. In the `__init__` method, we specify the types of layers we wanto to use as building blocks. The `forward` method specifies how these buidling blocks are connected.

In [16]:
class MLPModel(nn.Module):
    def __init__(self, input_dim):
        super(MLPModel, self).__init__()
        self.hidden = nn.Linear(input_dim, 256)
        self.output = nn.Linear(256, 1)
        self.relu = nn.ReLU()

    def forward(self, x):
        x = self.relu(self.hidden(x))
        x = self.output(x)
        return x

Let's create an instance of the neural network and look at its architecture.

In [17]:
input_dim = X_train.shape[1]
model = MLPModel(input_dim)


In [18]:
summary(model, input_size=(input_dim,))

----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Linear-1                  [-1, 256]          73,728
              ReLU-2                  [-1, 256]               0
            Linear-3                    [-1, 1]             257
Total params: 73,985
Trainable params: 73,985
Non-trainable params: 0
----------------------------------------------------------------
Input size (MB): 0.00
Forward/backward pass size (MB): 0.00
Params size (MB): 0.28
Estimated Total Size (MB): 0.29
----------------------------------------------------------------


### Training loop

We are now ready to train the neural network. We first specify the loss function, which is the mean squared error loss. We also specify the optimizer, which is the Adam optimizer. We then train the neural network for 100 epochs.

In [19]:
criterion = nn.MSELoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

In [20]:
epochs = 100

for epoch in range(epochs):
    model.train()
    epoch_loss = 0
    for batch_X, batch_y in train_loader:
        optimizer.zero_grad()
        predictions = model(batch_X)
        loss = criterion(predictions, batch_y)
        loss.backward()
        optimizer.step()
        epoch_loss += loss.item()

    if (epoch + 1) % 10 == 0:
        print(f'Epoch {epoch + 1}/{epochs}, Loss: {epoch_loss / len(train_loader):.4f}')


Epoch 10/100, Loss: 34962735104.0000
Epoch 20/100, Loss: 26745586286.7027
Epoch 30/100, Loss: 17235067329.7297
Epoch 40/100, Loss: 9488413107.8919
Epoch 50/100, Loss: 4621027295.1351
Epoch 60/100, Loss: 2444900914.5946
Epoch 70/100, Loss: 1754712652.3243
Epoch 80/100, Loss: 1480356024.6486
Epoch 90/100, Loss: 1337093276.9730
Epoch 100/100, Loss: 1216506246.9189


### Predictions and evaluation

Finally, we evaluate the neural network on the test set. Therefore, we set the model to evaluation mode, so that potential dropout layers are not active. We then compute the loss on the test set and print it.

In [21]:
# Evaluate the model
model.eval()
with torch.no_grad():
    y_pred = model(X_test_tensor)
    test_loss = criterion(y_pred, y_test_tensor)
    print(f'Test Loss: {test_loss.item():.4f}')


Test Loss: 1536884352.0000


Note that the loss is the MSE! Let's compute R2 and RMSE as well.

In [22]:
r2_test = r2_score(y_test_tensor, y_pred)
rmse_test = root_mean_squared_error(y_test_tensor, y_pred)
print('R2 on test set:', round(r2_test, 2))
print('RMSE on test set:', round(rmse_test, 2))

R2 on test set: 0.81
RMSE on test set: 39203.12


## Your turn!

The above RMSE is not very good. Try to improve it! You can try the following:

- Increase the number of epochs
- Increase the number of neurons in the hidden layer
- Add more hidden layers
- Add dropout layers
- ...

In [23]:
model = MLPModel(input_dim)
criterion = nn.MSELoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)
epochs = 1000

for epoch in range(epochs):
    model.train()
    epoch_loss = 0
    for batch_X, batch_y in train_loader:
        optimizer.zero_grad()
        predictions = model(batch_X)
        loss = criterion(predictions, batch_y)
        loss.backward()
        optimizer.step()
        epoch_loss += loss.item()

    if (epoch + 1) % 100 == 0:
        print(f'Epoch {epoch + 1}/{epochs}, Loss: {epoch_loss / len(train_loader):.4f}')

# Evaluate the model
model.eval()
with torch.no_grad():
    y_pred = model(X_test_tensor)
    test_loss = criterion(y_pred, y_test_tensor)
    print(f'Test Loss: {test_loss.item():.4f}')

r2_test = r2_score(y_test_tensor, y_pred)
rmse_test = root_mean_squared_error(y_test_tensor, y_pred)
print('R2 on test set:', round(r2_test, 2))
print('RMSE on test set:', round(rmse_test, 2))

Epoch 100/1000, Loss: 838001410.8108
Epoch 200/1000, Loss: 730465046.2703
Epoch 300/1000, Loss: 677295186.8108
Epoch 400/1000, Loss: 646932015.1351
Epoch 500/1000, Loss: 624843812.1081
Epoch 600/1000, Loss: 613584400.6486
Epoch 700/1000, Loss: 598359602.2703
Epoch 800/1000, Loss: 592041806.2703
Epoch 900/1000, Loss: 583451142.7027
Epoch 1000/1000, Loss: 575328038.7027


In [28]:
class MLPModel(nn.Module):
    def __init__(self, input_dim):
        super(MLPModel, self).__init__()
        self.hidden = nn.Linear(input_dim, 256)
        self.hidden2 = nn.Linear(256, 128)
        self.output = nn.Linear(128, 1)
        self.relu = nn.ReLU()
        self.dropout = nn.Dropout(0.2)

    def forward(self, x):
        x = self.relu(self.hidden(x))
        x = self.dropout(x)
        x = self.relu(self.hidden2(x))
        x = self.dropout(x)
        x = self.output(x)
        return x

# Instantiate the model
model = MLPModel(input_dim)
summary(model, input_size=(input_dim,))

----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Linear-1                  [-1, 256]          73,728
              ReLU-2                  [-1, 256]               0
           Dropout-3                  [-1, 256]               0
            Linear-4                  [-1, 128]          32,896
              ReLU-5                  [-1, 128]               0
           Dropout-6                  [-1, 128]               0
            Linear-7                    [-1, 1]             129
Total params: 106,753
Trainable params: 106,753
Non-trainable params: 0
----------------------------------------------------------------
Input size (MB): 0.00
Forward/backward pass size (MB): 0.01
Params size (MB): 0.41
Estimated Total Size (MB): 0.42
----------------------------------------------------------------


In [29]:
# Loss and optimizer
criterion = nn.MSELoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

# Training loop
epochs = 1500  # Increased epochs
for epoch in range(epochs):
    model.train()
    epoch_loss = 0
    for batch_X, batch_y in train_loader:
        optimizer.zero_grad()
        predictions = model(batch_X)
        loss = criterion(predictions, batch_y)
        loss.backward()
        optimizer.step()
        epoch_loss += loss.item()

    if (epoch + 1) % 100 == 0:
        print(f'Epoch {epoch + 1}/{epochs}, Loss: {epoch_loss / len(train_loader):.4f}')

# Evaluate the model
model.eval()
with torch.no_grad():
    y_pred = model(X_test_tensor)
    test_loss = criterion(y_pred, y_test_tensor)
    print(f'Test Loss: {test_loss.item():.4f}')

# Metrics
r2_test = r2_score(y_test_tensor.numpy(), y_pred.numpy())
rmse_test = root_mean_squared_error(y_test_tensor.numpy(), y_pred.numpy())
print('R2 on test set:', round(r2_test, 2))
print('RMSE on test set:', round(rmse_test, 2))

Epoch 100/1500, Loss: 770012321.0811
Epoch 200/1500, Loss: 704672060.3243
Epoch 300/1500, Loss: 649857196.7568


KeyboardInterrupt: 